In [1]:
import pandas as pd

# = = = = = = = = = = = = = = =

from urllib.parse import urlparse, parse_qs, urlencode

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

import re
import html as html_lib

# = = = = = = = = = = = = = = =

import math

In [2]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/input.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：2



,No,Url
0,1,https://toyota.oempartsonline.com/search?searc...
1,2,https://nissan.oempartsonline.com/search?searc...


In [3]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                     1
 Url    https://toyota.oempartsonline.com/search?searc...
 Name: 0, dtype: object)

In [4]:
import time

url_parse = urlparse(crawler_series['Url'])

dict_param = parse_qs(url_parse.query)
dict_param.pop('page', None)

url_base = f'{url_parse.scheme}://{url_parse.netloc}{url_parse.path}?{urlencode(dict_param, doseq=True)}'

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_request = url_base

        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            result = etree.tostring(html.xpath('//h1[@data-test-id="page-subheader__page_heading"]')[0]).decode('utf-8')
            result = re.sub(r'<.*?>', '', result)
            result = re.sub(r' +', ' ', result.strip())
            result = html_lib.unescape(result)

            count = int(result.split('Results')[0].replace(',', '').replace(' ', ''))
            if 'results for' in result.lower() and dict_param['search_str'][0].lower().strip() in result.lower() and count > 0:
                with open('./page.html', 'w', encoding='UTF-8') as file:
                    file.write(str(soup))
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

result, count

1


('1,183 Results for “Resonator”', 1183)

In [5]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Url': url_request,
                          'SKU Count': count,
                          'Page': math.ceil(count / 18)})

list_dict_correct

[{'No': 1,
  'Url': 'https://toyota.oempartsonline.com/search?search_str=Resonator',
  'SKU Count': 1183,
  'Page': 66}]

In [6]:
crawler_status = True

crawler_status

True

In [7]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Url            https://toyota.oempartsonline.com/search?searc...
 Request_Url    https://toyota.oempartsonline.com/search?searc...
 Name: 0, dtype: object]

In [8]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_page.xlsx', index=False)

df_correct

,No,Url,SKU Count,Page
0,1,https://toyota.oempartsonline.com/search?searc...,1183,66


In [9]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_page_error.xlsx', index=False)

df_error

,No,Url,Request_Url
0,1,https://toyota.oempartsonline.com/search?searc...,https://toyota.oempartsonline.com/search?searc...
